In [1]:
from polls import *
from UKGE import *
import numpy as np
import datetime

In [2]:
run_sim(res_path="outputs/resultsclusteredconstituencies.csv",output_path="outputs/results/Results_")

BigStd:  0.11871846200612296


/home/john/Desktop/Politics/UKGE/UKGE.py:158: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rand_df[p+"Seats"] = rand_df[[c + "Winner" for c in constituencies]].eq(p).sum(1)
/home/john/Desktop/Politics/UKGE/UKGE.py:158: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rand_df[p+"Seats"] = rand_df[[c + "Winner" for c in constituencies]].eq(p).sum(1)
/home/john/Desktop/Politics/UKGE/UKGE.py:158: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

In [36]:
results_df

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
Aberafan MaestegconFinal,0.045443,0.027779,0.03477,0.0,0.10544,0.066375,0.049491,0.0,0.085733,0.036173,...,0.0,0.0,0.019812,0.104806,0.012357,0.117664,0.0,0.10625,0.069185,0.0
AldershotconFinal,0.438884,0.524461,0.375423,0.423394,0.465716,0.468523,0.322528,0.30663,0.413358,0.373289,...,0.377802,0.370099,0.402404,0.447242,0.274693,0.448499,0.358082,0.505758,0.443586,0.211636
Aldridge-BrownhillsconFinal,0.438952,0.56579,0.474871,0.538541,0.529865,0.462089,0.374559,0.350932,0.476058,0.451801,...,0.417142,0.453221,0.430611,0.512835,0.353018,0.557215,0.417696,0.528866,0.582482,0.303341
Altrincham and Sale WestconFinal,0.270063,0.376026,0.243621,0.293492,0.343611,0.329895,0.219659,0.202519,0.296503,0.243495,...,0.27762,0.213105,0.253023,0.352569,0.176106,0.378687,0.217759,0.373343,0.312203,0.08706
Alyn and DeesideconFinal,0.191356,0.186213,0.179537,0.178766,0.263837,0.231812,0.159345,0.107826,0.213368,0.20641,...,0.132136,0.177529,0.168599,0.249075,0.144098,0.274263,0.099315,0.328101,0.268996,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
libSeats,23,19,38,29,10,14,27,30,19,22,...,24,60,45,21,43,16,53,18,17,44
refSeats,0,0,1,0,0,0,0,0,0,0,...,0,0,3,2,0,0,0,0,3,0
natSeats,42,56,59,56,55,47,52,34,58,43,...,21,46,53,59,52,41,46,41,57,59
othSeats,1,0,2,1,1,1,2,2,0,2,...,1,2,4,2,2,0,1,0,2,3
